In [15]:

import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf


In [16]:

soybean_large = fetch_ucirepo(id=90)

# data (as pandas dataframes)
X = soybean_large.data.features
y = soybean_large.data.targets


In [ ]:

# Substituir '?' por NaN
X = X.replace('?', np.nan)

# Preencher valores ausentes com o valor mais frequente de cada coluna (moda)
for col in X.columns:
    X[col] = X[col].fillna(X[col].mode()[0])


C:\Users\masar\AppData\Local\Temp\ipykernel_16160\3565150298.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(X[col].mode()[0], inplace=True)
C:\Users\masar\AppData\Local\Temp\ipykernel_16160\3565150298.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [18]:

encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)


In [19]:

le = LabelEncoder()
y_encoded = le.fit_transform(y)


c:\Users\masar\Documents\UFF\IAVerde\ia_verde\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stop], verbose=1)


c:\Users\masar\Documents\UFF\IAVerde\ia_verde\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.1781 - loss: 2.7813 - val_accuracy: 0.3023 - val_loss: 2.5778
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3257 - loss: 2.4682 - val_accuracy: 0.3721 - val_loss: 2.3587
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4000 - loss: 2.2057 - val_accuracy: 0.4186 - val_loss: 2.1711
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4094 - loss: 1.9417 - val_accuracy: 0.5581 - val_loss: 1.9845
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5184 - loss: 1.7564 - val_accuracy: 0.5814 - val_loss: 1.7828
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6227 - loss: 1.5763 - val_accuracy: 0.6047 - val_loss: 1.5773
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6860 - loss: 1.3715 - val_accuracy: 0.6744 - val_loss: 1.4151
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7799 - loss: 1.1081 - val_accuracy: 0.6744 - val_loss: 1.2623

In [21]:

y_pred = model.predict(X_test).argmax(axis=1)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step 
Relatório de Classificação:


ValueError: Number of classes, 18, does not match size of target_names, 19. Try specifying the labels parameter